In [10]:
%load_ext autoreload
%autoreload 2
%load_ext autotime
%config InlineBackend.figure_format = 'retina'

import itertools
import os
from collections import defaultdict
from pathlib import Path
from sklearn.model_selection import KFold
from toolz import valmap
from typing import Tuple

for folder in itertools.chain([Path.cwd()], Path.cwd().parents):
    if (folder / 'Pipfile').exists():
        os.chdir(folder)
        break

from notebooks.heart_transplant.dependencies.heart_transplant_data import get_rolling_cv_inputs_cached
from notebooks.heart_transplant.dependencies.heart_transplant_functions import get_rolling_cv, \
    remove_missing_columns
from notebooks.heart_transplant.dependencies.heart_transplant_pipelines import get_transform, \
    get_xgboost_pipeline, get_random_forest_pipeline

from evaluation_functions import get_1_class_y_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 8.2 ms (started: 2021-03-25 12:40:03 +00:00)


In [7]:
_, y, X, dataset_raw, sampling_sets = get_rolling_cv_inputs_cached()

[Memory]2390.7s, 39.8min: Loading get_rolling_cv_inputs...
time: 1.63 s (started: 2021-03-25 12:32:21 +00:00)


In [ ]:
results_all = defaultdict(dict)

In [11]:
for train_set, test_set in sampling_sets:
    print('.')
    X_train = X.iloc[train_set].assign(tx_year=dataset_raw['tx_year']).sort_values(by='tx_year').drop(columns='tx_year')
    y_train = y.loc[X_train.index]

    X_train_with_year = X_train.assign(tx_year=dataset_raw['tx_year'])

    min_year = X_train_with_year['tx_year'].min()
    max_year = X_train_with_year['tx_year'].max()

    X_train_weights = [ (individual['tx_year']-min_year) / (max_year-min_year) for _, individual in X_train_with_year.iterrows()]

    X_test = X.iloc[test_set]
    y_test = y.loc[X_test.index]

    model = get_random_forest_pipeline(X, y)

    model.fit(X_train, y_train, classifier__sample_weight=X_train_weights)
    y_score = get_1_class_y_score(DataFrame(model.predict_proba(X_test)))
    roc_auc = roc_auc_score(y_test, y_score)
    results_all['weighted'][max_year] = roc_auc


    model.fit(X_train, y_train)
    y_score = get_1_class_y_score(DataFrame(model.predict_proba(X_test)))
    roc_auc = roc_auc_score(y_test, y_score)
    results_all['unweighted'][max_year] = roc_auc


.


NameError: name 'roc_auc_score' is not defined

time: 33.5 s (started: 2021-03-25 12:40:05 +00:00)


**Random forest ROC AUC**

In [ ]:
print(dict_to_table_vertical(valmap(lambda i: np.mean(list(i.values())), results_all)))